# P2. Les vues SQL

## Définition d'une vue SQL  

Une vue n'est rien de plus qu'une instruction SQLite qui est stockée dans la base de données avec un nom associé. Il s'agit en fait d'une composition d'une table sous la forme d'une requête SQLite prédéfinie.

Une vue peut contenir toutes les lignes d'une table ou des lignes sélectionnées d'une ou plusieurs tables. Une vue peut être créée à partir d'une ou plusieurs tables qui dépendent de la requête SQLite écrite pour créer une vue.

## Code utilitaire

In [1]:
import os
import sqlite3
import pandas as pd   

In [2]:
from pathlib import Path

In [3]:
# Ouvre connection vers SQLite db
chemin_bdd = Path('db') / 'chinook.db'
conn = sqlite3.connect(chemin_bdd)

In [4]:
# Fonction permettant d'exécuter un requête SQL sur une BDD définie par sa connexion conn
def executer_requete(requete_sql, conn):
    try:
        cursor = conn.cursor()
        cursor.execute(requete_sql)
        conn.commit()
    except sqlite3.Error as e:
        print("Erreur lors de l'execution de la requête")
        print(e)
        return
    cursor.close()


## Example de vue SQL: v_albums

La requête suivante permet d'obtenir une liste (simplifiée) des albums / artistes correspondants:  

In [5]:
# Requete sur les tables albums & artists
requete = """
    SELECT AlbumId,
           Title,
           a.Name AS Artist
    FROM albums
    JOIN artists a on albums.ArtistId = a.ArtistId
    ORDER BY Artist;
"""

albums = pd.read_sql_query(requete, conn)
albums.head(5)

,AlbumId,Title,Artist
0,1,For Those About To Rock We Salute You,AC/DC
1,4,Let There Be Rock,AC/DC
2,296,"A Copland Celebration, Vol. I",Aaron Copland & London Symphony Orchestra
3,267,Worlds,Aaron Goldberg
4,280,The World of Classical Favourites,Academy of St. Martin in the Fields & Sir Nevi...


Le code ci-dessous permet de créer une vue correspondant à la requête ci-dessous :    
Référence vue sous SQLite : https://www.sqlite.org/lang_createview.html

In [6]:
# creation vue albums avec artistes
creation_vue_albums = """
    CREATE VIEW v_albums (
        AlbumId,
        Title,
        Artist)
    AS
        SELECT AlbumId,
               Title,
               a.Name AS Artist
        FROM albums
        JOIN artists a on albums.ArtistId = a.ArtistId
        ORDER BY Artist, Title;
"""

executer_requete(creation_vue_albums, conn)

Une fois la vue créé, on peut la réutiliser dans des requêtes SQL comme une table:  

In [7]:
# Utilisation de la vue v_albums
albums = pd.read_sql_query("SELECT * FROM v_albums LIMIT 10;", conn)
albums.head(5)

,AlbumId,Title,Artist
0,1,For Those About To Rock We Salute You,AC/DC
1,4,Let There Be Rock,AC/DC
2,296,"A Copland Celebration, Vol. I",Aaron Copland & London Symphony Orchestra
3,267,Worlds,Aaron Goldberg
4,280,The World of Classical Favourites,Academy of St. Martin in the Fields & Sir Nevi...


In [8]:
# Utilisation de la vue v_albums avec une clause WHERE
albums = pd.read_sql_query("SELECT * FROM v_albums WHERE Artist = 'U2';", conn)
albums

,AlbumId,Title,Artist
0,232,Achtung Baby,U2
1,233,All That You Can't Leave Behind,U2
2,234,B-Sides 1980-1990,U2
3,235,How To Dismantle An Atomic Bomb,U2
4,255,Instant Karma: The Amnesty International Campa...,U2
5,236,Pop,U2
6,237,Rattle And Hum,U2
7,238,The Best Of 1980-1990,U2
8,239,War,U2
9,240,Zooropa,U2


In [9]:
# Utilisation de la vue v_albums avec une autre jointure
requete = """
    SELECT a.AlbumId,
           a.Title,
           a.Artist,
           count(t.TrackId) as Tracks
    FROM v_albums a
    JOIN tracks t ON a.AlbumId = t.AlbumId
    GROUP BY 1, 2, 3
    ORDER BY 3, 2;
"""
albums = pd.read_sql_query(requete, conn)
albums.head()

,AlbumId,Title,Artist,Tracks
0,1,For Those About To Rock We Salute You,AC/DC,10
1,4,Let There Be Rock,AC/DC,8
2,296,"A Copland Celebration, Vol. I",Aaron Copland & London Symphony Orchestra,1
3,267,Worlds,Aaron Goldberg,1
4,280,The World of Classical Favourites,Academy of St. Martin in the Fields & Sir Nevi...,2


In [10]:
# Effacer une vue
executer_requete("DROP VIEW IF EXISTS v_albums;", conn)

### Question: Avantages & Inconvénients des vues SQL

Quelles sont les avantages et inconvénients des vues SQL ?  
*A compléter*

---

## Exercices de création / mise en oeuvre de vues

### Vue 1: caractéristiques complètes des Albums  
A partir de l'exemple de vue ci-dessus, créer et exploiter une nouvelle vue v_albums reprenant pour chaque album:  
- AlbumId
- Title
- Artist
- nb de pistes
- Durée totale de l'album (en minutes)
- Prix de l'album

### Vue 2: caractéristiques détaillées des pistes
Créer et exploiter une nouvelle vue v_tracks reprenant pour chaque piste:  
- TrackId
- Name
- Genre
- Album
- Artist
- Composer
- Temps (en seconde)
- MediaType

### Vue 3: Vue augmentée invoices
A. Créer une nouvelle vue v_invoices à partir de la table invoices ajoutant 3 nouveaux attributs basés sur l'attribut `InvoiceDate`:  
- semaine de la transaction
- mois de la transaction
- année de la transaction

B. Exploiter la vue v_invoices ainsi crée pour déterminer les ventes annuelles par pays.

### Vue 4: A vous de jouer
Créer et exploiter une nouvelle vue de votre choix  